In [1]:
from pymongo import MongoClient


# 1. Kết nối

| Parameter | Description |
|-----------|-------------|
| username | Tài khoản username |
| password | Mật khẩu của username|
| authSource | DB để login |
| connectTimeoutMS | Timeout kết nối |
| serverSelectionTimeoutMS | Timeout chọn node |
| socketTimeoutMS | Timeout đọc/ghi |
| maxPoolSize | Kết nối tối đa |
| minPoolSize | Giữ sẵn kết nối |
| maxIdleTimeMS | Đóng connection idle |
| retryReads | Retry đọc |
| retryWrites | Retry ghi |
| w="majority" | Ghi an toàn |
| read_preference | None |
| write_concern | None |
| read_concern | None |

In [2]:
mongo_client = MongoClient(host="localhost",
                           port=27017,
                           username="duc_le",
                           password="ducle1908",
                           authSource="admin"
                           
)

In [3]:
database = mongo_client.get_database(name="test")
collection = database.get_collection(name="test")

In [15]:
full = collection.find({},{"age":1})
for doc in full:
    print(doc)

{'_id': ObjectId('693d459604c2e9d3fc744bcf'), 'age': 21}
{'_id': ObjectId('693d460404c2e9d3fc744bd0'), 'age': 21}
